1. Imports required modules and sets seeds to 0 (for reproducibility)

In [ ]:
# Imports required modules
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

import torch.optim as optim
from torch.autograd import Variable

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, confusion_matrix, classification_report
import random

from PIL import Image
from os import listdir
from os.path import isfile, join

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

2. Sets the device for program execution

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

REMOVE LATER. Unzips the dataset. Not needed if the uncomprressed "final_dataset" folder is present in the same directory as the project notebook

In [ ]:
!unzip final_dataset.zip

3. Defines a function for preprocessing and loading data for the project

In [ ]:
def project_data_loader(project_dataset_path="./final_dataset"):
  """This function gets the project dataset from the provided path. The project 
  data is divided into training, validation, and testing datasets. Finally, the
  DataLoader objects are created for each of the three datasets.

  Arguments
  ---------
  project_dataset_path: str
    String representing the path to the project dataset directory

  Returns
  ---------
  train_dataloader: torch.utils.data.DataLoader
    DataLoader for training dataset
  valid_dataloader: torch.utils.data.DataLoader
    DataLoader for validation dataset
  test_dataloader: torch.utils.data.DataLoader
    DataLoader for testing dataset
  """

  # Defines image transform required for preprocessing
  preprocess_transform = transforms.Compose(
                   [transforms.Resize((32, 32)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  
  # Loads the project dataset
  project_dataset = ImageFolder(root=project_dataset_path,
                              transform=preprocess_transform)
  
  # Defines mappings between classes and labels
  project_dataset.class_to_idx = {"1_cloth_mask": 0,
                                "2_surgical": 1,
                                "3_n95": 2,
                                "4_incorrect_mask": 3,
                                "5_no_mask": 4}

  label_to_class = {v:k for k,v in project_dataset.class_to_idx.items()}

  # Automatically splits the project dataset into training, validation, and testing datasets
  tvt_count = len(project_dataset)
  tv_count = int(tvt_count * 0.8)
  train_valid_data, test_data = random_split(project_dataset, 
                                             [tv_count, tvt_count - tv_count])
  tv_count = len(train_valid_data)
  train_count = int(tv_count * 0.8)
  train_data, valid_data = random_split(train_valid_data, 
                                        [train_count, tv_count - train_count])
  
  # Creates dataloaders for training, validation, and testing data
  train_dataloader = DataLoader(train_data, batch_size=64)
  valid_dataloader = DataLoader(valid_data, batch_size=64)
  test_dataloader = DataLoader(test_data, batch_size=64)

  return train_dataloader, valid_dataloader, test_dataloader, label_to_class

4. Defines the CNN architecture

4.1 Final architecture: Simple CNN architecture

In [ ]:
class StackedCNN(nn.Module):
    def __init__(self):
        super(StackedCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=8,
                               kernel_size=3,
                               padding=3//2)
        
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=8,
                               out_channels=16,
                               kernel_size=3,
                               padding=3//2)

        self.linear1 = nn.Linear(16 * 8 * 8, 256)
        self.linear2 = nn.Linear(256, 64)
        self.linear3 = nn.Linear(64, 5)

        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(F.relu(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

4.2 Variant 1: Pooling layers removed from the final architecture

In [ ]:
class StackedCNNv1(nn.Module):
    def __init__(self):
        super(StackedCNNv1, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=8,
                               kernel_size=3,
                               padding=3//2)
        
        self.conv2 = nn.Conv2d(in_channels=8,
                               out_channels=16,
                               kernel_size=3,
                               padding=3//2)

        self.linear1 = nn.Linear(16384, 256)
        self.linear2 = nn.Linear(256, 64)
        self.linear3 = nn.Linear(64, 5)

        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

4.3 Variant 2: Different number of convolution layers from the final architecture - 3 convolution layers

In [ ]:
class StackedCNNv2(nn.Module):
    def __init__(self):
        super(StackedCNNv2, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=8,
                               kernel_size=3,
                               padding=3//2)
        
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=8,
                               out_channels=16,
                               kernel_size=3,
                               padding=3//2)
        
        self.conv3 = nn.Conv2d(in_channels=16,
                               out_channels=32,
                               kernel_size=3,
                               padding=3//2)
        
        self.linear1 = nn.Linear(32 * 4 * 4, 256)
        self.linear2 = nn.Linear(256, 64)
        self.linear3 = nn.Linear(64, 5)

        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool1(F.relu(self.conv2(x)))
        x = self.pool1(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

5. Defines a function for training the neural network

In [ ]:
def train_model(train_dataloader, valid_dataloader, model, criterion, optimizer, num_epoch=20, title="current_model", save_dir="./saved_model"):

  training_losses = list()
  validation_losses = list()

  prev_loss = np.inf
  tolerance = 3
  loss_counter = 0

  for epoch in range(num_epoch):  # Process the dataset "num_epoch" times
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        # "data" is a list of [inputs, labels]
        # Get the inputs and labels from data
        inputs, labels = data

        # Setting the device for inputs and labels
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()               # Resets all parameter gradients to zero

        outputs = model(inputs)             # Completes a forward pass through the network
        loss = criterion(outputs, labels)   # Calculates loss of predictions with respect to tagets
        loss.backward()                     # Calculates gradient of loss with respect to all parameters
        optimizer.step()                    # Performs optimization step using the calculated gradients

    # Prints loss value after every epoch
    # print("Epoch %03d: Train_loss: %.4f " %(epoch+1, loss.item()))
    training_losses.append(loss.item())

    
    model.eval()
    with torch.no_grad():
      validation_iterator = iter(valid_dataloader)
      valid_inputs, valid_labels = validation_iterator.next()

      valid_inputs = valid_inputs.to(device)
      valid_labels = valid_labels.to(device)
      
      valid_outputs = model(valid_inputs)
      valid_loss = criterion(valid_outputs, valid_labels)
      print("Epoch %03d: Train_loss: %.4f, Validation_loss: %.4f " %(epoch+1, loss.item(), valid_loss.item()))
      validation_losses.append(valid_loss.item())

    # Stops training before completing the number of epochs if the validation loss gets worse for "tolerance" times
    if valid_loss > prev_loss:
      loss_counter += 1
    else:
      loss_counter = 0
    
    if loss_counter >= tolerance:
      print("Observing increasing values for validation loss. Early stopping.")
      break
    
    prev_loss = valid_loss

  plt.plot(training_losses, label="Training loss")
  plt.plot(validation_losses, label="Validation loss")
  plt.title("Loss vs epoch for " + title)
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend(frameon=False)
  plt.show()

  print("Training completed. Saving model.")
  MODEL_SAVE_PATH = save_dir
  torch.save(model.state_dict(), MODEL_SAVE_PATH)
  return MODEL_SAVE_PATH


6. Defines a function for loading saved model and testing

In [ ]:
def plot_confusion_matrix(y_preds, y_true, labels, title):
    """This function displays the confusion matrix for provided predictions by a classifier.

    Arguments
    ---------
    y_preds : numpy.ndarray
      Predictions by a classifier
    y_true : numpy.ndarray
      Labels corresponding to the predictions in y_preds
    labels : list
      Label names to be used for the confusion matrix
    title : str
      Title to be used for the confusion matrix

    """

    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fix, ax = plt.subplots(figsize=(6, 6))
    cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    cm_display.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix for " + title)
    plt.show()


def print_classification_report(y_preds, y_true, title):
    """This function displays the classification report for provided predictions by a classifier.

    Arguments
    ---------
    y_preds : numpy.ndarray
      Predictions by a classifier
    y_true : numpy.ndarray
      Labels corresponding to the predictions in y_preds
    title : str
      Title to be used for the confusion matrix

    """

    labels = ["0_cloth", "1_surgical", "2_n95", "3_incorrect", "4_no_mask"]
    plot_confusion_matrix(y_preds, y_true, labels, title)
    print("\n")
    print("Classification report for " + title)
    acc = accuracy_score(y_true, y_preds)
    f1 = f1_score(y_true, y_preds, average="weighted")
    prec = precision_score(y_true, y_preds, average="weighted")
    rec = recall_score(y_true, y_preds, average="weighted")

    print("-" * 75)
    print("Accuracy: %.2f%%" % (acc * 100))
    print("Precision: %.2f%%" % (prec * 100))
    print("Recall: %.2f%%" % (rec * 100))
    print("F1 score: %.2f%%" % (f1 * 100))
    print("-" * 75)
    print("\n\n")

In [ ]:
def test_model(model, test_dataloader, title, model_load_path="./saved_model"):

  print("Testing model performance for the entire test dataset: ")
  # Loads test dataset images
  test_iterator = iter(test_dataloader)
  inputs, labels = test_iterator.next()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # Loads saved model
  model.load_state_dict(torch.load(model_load_path))
  model.to(device)

  # Gets model outputs
  model.eval()
  outputs = model(inputs)
  _, predicted = torch.max(outputs, 1)

  preds = predicted.to(torch.device("cpu")).numpy()
  targets = labels.to(torch.device("cpu")).numpy()

  print_classification_report(preds, targets, title)

  # matrix = confusion_matrix(targets, preds)
  # print(matrix)

  report = classification_report(targets, preds, target_names=["0_cloth", "1_surgical", "2_n95", "3_incorrect", "4_no_mask"])
  print(report) 

7. Defines a function for using the model for single input prediction (application mode). This function only considers the final CNN architecture. It does not consider the variant models.

In [ ]:
def single_prediction(image, model_load_path="./saved_model"):

  label_to_class = {0: '1_cloth_mask', 1: '2_surgical', 2: '3_n95', 3: '4_incorrect_mask', 4: '5_no_mask'}
  preprocess_transform = transforms.Compose(
                   [transforms.Resize((32,32)),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
  image_tensor = preprocess_transform(image).float()
  image_tensor = image_tensor.unsqueeze_(0)
  input = Variable(image_tensor)
  input = input.to(device)

  # Loads saved model
  loaded_model = StackedCNN()
  loaded_model.load_state_dict(torch.load(model_load_path))

  loaded_model.to(device)
  # Gets model outputs
  loaded_model.eval()

  output = loaded_model(input)
  _, prediction = torch.max(output, 1)

  return "Classification: " + label_to_class[prediction.item()]

8. Defines setup for experiments:
Defines variables for the model, hyperparameters, Classification Cross-Entropy loss and SGD with momentum

In [ ]:
"""
Please uncomment and use the models as required.
"""

# Final CNN architecture for the project
model = StackedCNN()
model_name = "stacked_cnn"
model_save_dir = "./stacked_cnn"

# Variant 1
# model = StackedCNNv1()
# model_name = "variant_1"
# model_save_dir = "./variant_1"

# Variant 2
# model = StackedCNNv2()
# model_name = "variant_2"
# model_save_dir = "./variant_2"

model.to(device)

# Sets hyperparameter values
num_epoch = 150

class_samples = [445, 599, 209, 706, 641]
class_weights = [1 - (x / sum(class_samples)) for x in class_samples]
class_weights = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

9. Runs experiments

In [ ]:
# Loads dataset and splits into training, validation, and testing dataset
train_dataloader, valid_dataloader, test_dataloader, label_to_class = project_data_loader()

# Trains and saves the model
saved_model_path = train_model(train_dataloader, valid_dataloader, model, criterion, optimizer, num_epoch=num_epoch, title=model_name, save_dir=model_save_dir)

# Loads saved model and tests the performance using test dataset
test_model(model, test_dataloader, title=model_name, model_load_path=saved_model_path)
print("Testing completed.")
print("-" * 75)

# Loads saved model and performs prediction for single input for 5 samples
print("Executing single input predition for 5 sample images.")
SINGLE_INPUT_DATA = "./app_mode_samples"
sample_inputs = [join(SINGLE_INPUT_DATA, f) for f in listdir(SINGLE_INPUT_DATA) if isfile(join(SINGLE_INPUT_DATA, f))]
for sample_input in sample_inputs:
  with Image.open(sample_input) as image:
    plt.figure()
    plt.imshow(np.asarray(image))
    plt.title(single_prediction(image))
